In [ ]:
# %%
from ProtDiffusion.training_utils import VAETrainingConfig, make_clustered_dataloader, set_seed, VAETrainer, count_parameters
from transformers import PreTrainedTokenizerFast

from datasets import load_from_disk

from ProtDiffusion.models.autoencoder_kl_1d import AutoencoderKL1D

import os

seed = 42
set_seed(seed) # Set the random seed for reproducibility

dataset = load_from_disk('/home/kkj/ProtDiffusion/datasets/UniRef50_grouped')
# dataset = load_from_disk('/home/kkj/ProtDiffusion/datasets/ACP-bad?_grouped')
dataset = dataset.shuffle(seed)

# %%
# tokenizer = PreTrainedTokenizerFast.from_pretrained("/zhome/fb/0/155603/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.1")
# tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.1")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.2")

# Split the dataset into train and temp sets using the datasets library
train_test_split_ratio = 0.0002
train_val_test_split = dataset.train_test_split(test_size=train_test_split_ratio, seed=seed)
train_dataset = train_val_test_split['train']
temp_dataset = train_val_test_split['test']

# Split the temp set into validation and test sets using the datasets library
val_test_split_ratio = 0.5
val_test_split = temp_dataset.train_test_split(test_size=val_test_split_ratio, seed=seed)
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

# Check dataset lengths
print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

In [ ]:
mega_batch = 100
batch_size = 16
max_len_start = 64
max_len = max_len_start

print("num cpu cores:", os.cpu_count())
print("setting num_workers to 16")
num_workers = 16

train_dataloader = make_clustered_dataloader(batch_size,
                                             mega_batch,
                                             train_dataset,
                                             tokenizer=tokenizer,
                                             max_len=max_len_start,
                                             num_workers=num_workers,
)
val_dataloader = make_clustered_dataloader(batch_size,
                                           mega_batch,
                                           val_dataset, 
                                           tokenizer=tokenizer,
                                           max_len=max_len, 
                                           num_workers=1,
)
test_dataloader = make_clustered_dataloader(batch_size,
                                            mega_batch,
                                            test_dataset,
                                            tokenizer=tokenizer,
                                            max_len=max_len, 
                                            num_workers=1,
)
print("length of train dataloader: ", len(train_dataloader))
print("length of val dataloader: ", len(val_dataloader))
print("length of test dataloader: ", len(test_dataloader))

In [ ]:
for i,batch in enumerate(train_dataloader):
    print(batch)
    if i == 10:
        break

In [ ]:
for epoch in range(1):
    for i,batch in enumerate(train_dataloader):
        print("batch", (epoch*len(train_dataloader)) + i)